In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [47]:
# 读取数据\
dtypes = {'creativeID': object, 'userID': object, 'connectionType': object, 'telecomsOperator': object}  # 指定分类特征
train = pd.read_csv('../data/train.csv', dtype=dtypes)
test = pd.read_csv('../data/test.csv', dtype=dtypes)


In [48]:
# 查看数据的前5行
# train.head()
# 查看数据的最后5行
train.tail()

,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator
9995,0,170114,NaN,4444,1413151,3498,1,1
9996,0,170114,NaN,1887,353167,3792,1,1
9997,0,170114,NaN,4565,1393358,6667,2,1
9998,0,170114,NaN,2137,1777707,2579,1,2
9999,1,170114,181032.0,2137,1746426,2579,1,1


In [49]:
test.head()

,instanceID,label,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,1,-1,310000,3745,1164848,3451,1,3
1,2,-1,310000,2284,2127247,1613,1,3
2,3,-1,310000,1456,2769125,5510,2,1
3,4,-1,310000,4565,9762,4113,2,3
4,5,-1,310000,49,2513636,3615,1,3


In [50]:
# 删除训练集中的conversionTime，因为测试集中没有这个特征
train = train.drop(['conversionTime'], axis=1)

In [51]:
train.head()

,label,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,0,170000,3089,2798058,293,1,1
1,0,170000,1259,463234,6161,1,2
2,0,170000,4465,1857485,7434,4,1
3,0,170000,1004,2038823,977,1,1
4,0,170000,1887,2015141,3688,1,1


In [52]:
# 统计每个特征取值非空的个数
train.count()

label               10000
clickTime           10000
creativeID          10000
userID              10000
positionID          10000
connectionType      10000
telecomsOperator    10000
dtype: int64

In [53]:
# 查看数据中是否有缺失值， False表示没有缺失值
train.isnull().any()

label               False
clickTime           False
creativeID          False
userID              False
positionID          False
connectionType      False
telecomsOperator    False
dtype: bool

In [54]:
# 如果存在缺失值的数据非常少，可以删除具有缺失值的行
train.dropna(inplace=True)

In [55]:
# 将label和instanceID单独拿出来
train_label = train['label']
test_label = test['label']
test_instanceID = test['instanceID']

# 从原始数据中删除label
train.drop(['label'], axis=1, inplace=True)
test.drop(['instanceID','label'], axis=1, inplace=True)

In [56]:
# 分类特征与数值特征
cat = train.dtypes[train.dtypes == 'object'].index  # 分类特征
num = train.dtypes[train.dtypes != 'object'].index  # 数值特征

In [57]:
cat

Index(['creativeID', 'userID', 'connectionType', 'telecomsOperator'], dtype='object')

In [58]:
num

Index(['clickTime', 'positionID'], dtype='object')

In [59]:
# 如果存在缺失值的数据很多，可以选择填充缺失数据
# 如果是分类特征，可以用该特征取值的众数填充，即出现次数最多的值
for col in cat:
    train.fillna(value=train[col].mode()[0], inplace=True)

# 如果是数值特征，可以用该特征特征的平均值填充
for col in num:
    train.fillna(value=train.mean(), inplace=True)

In [60]:
train.head()

,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,170000,3089,2798058,293,1,1
1,170000,1259,463234,6161,1,2
2,170000,4465,1857485,7434,4,1
3,170000,1004,2038823,977,1,1
4,170000,1887,2015141,3688,1,1


In [61]:
test.head()

,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,310000,3745,1164848,3451,1,3
1,310000,2284,2127247,1613,1,3
2,310000,1456,2769125,5510,2,1
3,310000,4565,9762,4113,2,3
4,310000,49,2513636,3615,1,3


In [62]:
# 连续值归一化
# train[num] = MinMaxScaler().fit_transform(train[num])
scale = MinMaxScaler()
scale.fit(train[num])
train[num] = scale.transform(train[num])
test[num] = scale.transform(test[num])

In [63]:
train[num].head()

,clickTime,positionID
0,0.0,0.035944
1,0.0,0.805720
2,0.0,0.972714
3,0.0,0.125672
4,0.0,0.481307


In [64]:
# one-hot编码
df_all = pd.concat([train, test], axis=0)  # 将train和test数据一起做one-hot编码
dummies = pd.get_dummies(df_all[cat], drop_first=True)  # 对分类特征one-hot编码
df_all = pd.concat([df_all, dummies], axis=1)  # 将原始数据和one-hot数据横向拼接起来
df_all.drop(cat, inplace=True, axis=1) # 删掉原始分类变量，留下连续值特征和one-hot编码特征

In [65]:
df_all.head()

,clickTime,positionID,creativeID_1040,creativeID_1052,creativeID_1064,creativeID_1069,creativeID_1073,creativeID_1086,creativeID_1087,creativeID_1092,...,userID_999852,userID_999953,userID_999969,connectionType_1,connectionType_2,connectionType_3,connectionType_4,telecomsOperator_1,telecomsOperator_2,telecomsOperator_3
0,0.0,0.035944,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,0.0,0.805720,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0.0,0.972714,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,0.0,0.125672,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,0.0,0.481307,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [66]:
train = df_all.iloc[:train.shape[0], :]
test = df_all.iloc[train.shape[0]:, :]

In [67]:
len(train)

10000

In [68]:
# 将label和instanceID重新放回数据
train['label'] = train_label
test['label'] = test_label
test['instanceID'] = test_instanceID

d:\program files\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
d:\program files\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
d:\program files\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [69]:
train.head()

,clickTime,positionID,creativeID_1040,creativeID_1052,creativeID_1064,creativeID_1069,creativeID_1073,creativeID_1086,creativeID_1087,creativeID_1092,...,userID_999953,userID_999969,connectionType_1,connectionType_2,connectionType_3,connectionType_4,telecomsOperator_1,telecomsOperator_2,telecomsOperator_3,label
0,0.0,0.035944,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,0.0,0.805720,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0.0,0.972714,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,0.0,0.125672,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0.0,0.481307,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [70]:
test.head()

,clickTime,positionID,creativeID_1040,creativeID_1052,creativeID_1064,creativeID_1069,creativeID_1073,creativeID_1086,creativeID_1087,creativeID_1092,...,userID_999969,connectionType_1,connectionType_2,connectionType_3,connectionType_4,telecomsOperator_1,telecomsOperator_2,telecomsOperator_3,label,instanceID
0,1228.070175,0.450216,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,-1,1
1,1228.070175,0.209104,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,-1,2
2,1228.070175,0.720320,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,-1,3
3,1228.070175,0.537059,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,-1,4
4,1228.070175,0.471730,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,-1,5


In [71]:
# 将新数据保存到csv文件中
train.to_csv('../data/train_processed.csv', sep=',', index=False, header=True)
test.to_csv('../data/test_processed.csv', sep=',', index=False, header=True)